In [1]:
import pickle
import json
import pandas as pd
from pandas.io.json import json_normalize

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

with open('outfile1', 'rb') as fp:
    baseball_data = pickle.load(fp)  # Load up the 2018 data

del baseball_data[-1] # Last day, no games
del baseball_data[108] # All star break, no games
del baseball_data[108] # All star break, no games
del baseball_data[108] # All star break, no games
del baseball_data[30] # April 31 does not exist
del baseball_data[91] # June 31 does not exist


In [2]:
baseball_json = []

for days in baseball_data:
    baseball_json.append(json.loads(days))
    
for days in baseball_json:
    del days['_comment']  # API pull had weird last key value pair, delete
    
baseball_json_norm = json_normalize(baseball_json)
baseball_json_norm = baseball_json_norm.drop(
    columns=['league.alias', 'league.date','league.id', 'league.name'])

data = json_normalize(baseball_json_norm.iloc[0,0])
for rows in range(1,180):
    data = data.append(json_normalize(baseball_json_norm.iloc[rows, 0]), 
                       ignore_index=True, sort=False)
    
data['home.win.pct'] = data['game.home.win'] / (data['game.home.win'] + data['game.home.loss'])
data['away.win.pct'] = data['game.away.win'] / (data['game.away.win'] + data['game.away.loss'])    
data['total.runs'] = data['game.home.runs'] + data['game.away.runs'] # Response variable!

In [3]:
with open('outfile_april_12_pre', 'rb') as fp:
    baseball_data1 = pickle.load(fp)

In [4]:
data_all_cols = list(data.columns.values)
data_good_cols = list(baseball_data1.columns.values)
data_good_cols.append('home.win.pct')
data_good_cols.append('away.win.pct')
data_good_cols.append('total.runs')
data = data.drop(list(set(data_all_cols) - set(data_good_cols)), axis=1)
# data contains only the 67 variables available before today's MLB games start

# Must drop additional variables will not know before game starts
data = data.drop(['game.away.errors', 'game.away.hits', 'game.away.runs',
                 'game.home.errors', 'game.home.hits', 'game.home.runs'], axis=1)

# Must drop unnecessary variables
data = data.drop(['game.away.id', 'game.away.market', 'game.away.name',
                 'game.away.probable_pitcher.first_name', 
                 'game.away.probable_pitcher.jersey_number',
                 'game.away.probable_pitcher.last_name',
                 'game.away.probable_pitcher.preferred_name',
                 'game.away_team', 'game.coverage',
                 'game.game_number', 'game.home.id', 
                 'game.home.market', 'game.home.name',
                 'game.home.probable_pitcher.first_name', 
                 'game.home.probable_pitcher.jersey_number',
                 'game.home.probable_pitcher.last_name',
                 'game.home.probable_pitcher.preferred_name',
                 'game.home_team', 'game.rescheduled',
                 'game.scheduled', 'game.status',
                 'game.venue.address', 'game.venue.country',
                 'game.venue.market', 'game.venue.state',
                 'game.venue.zip', 'game.weather.forecast.obs_time', 
                 'game.id', 'game.venue.city', 'game.venue.id',
                 'game.double_header', 'game.venue.location.lat',
                 'game.venue.location.lng'], axis=1)

data.drop_duplicates(inplace=True)
data = data.drop([373]) # Pesky row

# Pitcher records set to 0 if NaN
data['game.away.probable_pitcher.loss'].fillna(0, inplace=True)
data['game.away.probable_pitcher.win'].fillna(0, inplace=True)
data['game.home.probable_pitcher.loss'].fillna(0, inplace=True)
data['game.home.probable_pitcher.win'].fillna(0, inplace=True)

# Missing values replaced with "unknown" for text values
data['game.away.probable_pitcher.id'].fillna('unknown', inplace=True)
data['game.home.probable_pitcher.id'].fillna('unknown', inplace=True)
data['game.weather.forecast.condition'].fillna('unknown', inplace=True)
data['game.weather.forecast.wind.direction'].fillna('unknown', inplace=True)
data['game.venue.field_orientation'].fillna('unknown', inplace=True)
data['game.venue.stadium_type'].fillna('unknown', inplace=True)


# Weather data simplified
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition'].str.replace("moderate snow", "Snow", case = False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition'].str.replace("heavy snow", "Snow", case = False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition'].str.replace("moderate or heavy snow showers", "Snow", case = False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition'].str.replace("moderate or snow showers", "Snow", case = False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition'].str.replace("patchy snow", "Snow", case = False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition'].str.replace("patchy light snow", "Snow", case = False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition'].str.replace("light snow", "Snow", case = False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition'].str.replace("light snow, mist", "Snow", case = False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition'].str.replace("squalls", "Snow", case = False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition'].str.replace("snow, mist", "Snow", case = False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition'].str.replace("light sleet", "Snow", case = False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition'].str.replace("light drizzle", "Rain", case = False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition'].str.replace("patchy rain possible", "Rain", case = False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition'].str.replace("Light rain", "Rain", case = False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition'].str.replace("patchy light rain", "Rain", case = False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition'].str.replace("moderate rain", "Rain", case = False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition'].str.replace("light rain shower", "Rain", case = False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition'].str.replace("patchy light drizzle", "Rain", case = False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition'].str.replace("light rain, mist", "Rain", case = False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition'].str.replace("light drizzle, mist", "Rain", case = False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition'].str.replace("Shower in Vicinity", "Rain", case = False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition'].str.replace("overcast", "Cloudy", case = False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition'].str.replace("Thundery outbreaks possible", "Storm", case = False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition'].str.replace("rain shower", "Rain", case = False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition'].str.replace("thunderstorm in vicinity", "Storm", case = False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition'].str.replace("heavy rain", "Storm", case = False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition'].str.replace("torrential rain shower", "Storm", case = False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition'].str.replace("rain with thunderstorm, mist", "Storm", case = False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition'].str.replace("thunderstorm", "Storm", case = False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition'].str.replace("thunderstorm, haze", "Storm", case = False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition'].str.replace("heavy rain, mist", "Rain", case = False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition'].str.replace("Partly cloudy", "Cloudy", case = False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition'].str.replace("Patchy rain", "Rain", case = False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition'].str.replace("Rain, Mist", "Mist", case = False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition'].str.replace("Torrential Rain", "Storm", case = False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition'].str.replace("Moderate or Storm", "Storm", case = False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition'].str.replace("Patchy rain with thunder", "Storm", case = False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition'].str.replace("rain with storm", "Storm", case = False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition'].str.replace("storm, rain", "Storm", case = False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition'].str.replace("storm with storm", "Storm", case = False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition'].str.replace("Storm, Mist", "Mist", case = False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition'].str.replace("Rain with thunder", "Storm", case = False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition'].str.replace("Blowing Widespread Dust", "Haze", case = False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition'].str.replace("Smoke", "Haze", case = False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition'].str.replace("Storm, Haze", "Storm", case = False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition'].str.replace("Haze, Smoke", "Haze", case = False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition'].str.replace("Storm, Fog", "Storm", case = False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition'].str.replace("Haze, Haze", "Haze", case = False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition'].str.replace("Fog", "Mist", case = False)

In [5]:
data.head(7)

,game.away.abbr,game.away.loss,game.away.probable_pitcher.era,game.away.probable_pitcher.id,game.away.probable_pitcher.loss,game.away.probable_pitcher.win,game.away.win,game.broadcast.network,game.day_night,game.home.abbr,game.home.loss,game.home.probable_pitcher.id,game.home.win,game.venue.capacity,game.venue.field_orientation,game.venue.name,game.venue.stadium_type,game.venue.surface,game.weather.forecast.cloud_cover,game.weather.forecast.condition,game.weather.forecast.dew_point_f,game.weather.forecast.humidity,game.weather.forecast.temp_f,game.weather.forecast.wind.direction,game.weather.forecast.wind.speed_mph,game.home.probable_pitcher.era,game.home.probable_pitcher.loss,game.home.probable_pitcher.win,home.win.pct,away.win.pct,total.runs
0,CWS,16,6.0,65b0e836-7f64-4ecc-8942-db8a90ba7a98,1.0,1.0,7,FS-KC,D,KC,19,1019e26a-57a1-4480-aed5-843bab37154a,5,37903,NE,Kauffman Stadium,outdoor,grass,0.0,Sunny,34.0,31.0,67.0,ENE,9.0,NaN,0.0,0.0,0.208333,0.304348,8
1,NYY,1,NaN,b004cc6c-f00c-4b5d-917a-ee1abf92179d,0.0,0.0,2,YES,D,TOR,2,cc86d4d3-1618-415e-b7fc-a303f3b8dd6f,1,53506,N,Rogers Centre,retractable,turf,94.0,Snow,23.0,60.0,35.0,WNW,14.0,NaN,0.0,0.0,0.333333,0.666667,11
2,STL,2,NaN,60b7e25e-7765-4d8b-82a6-d33e7179dae5,0.0,0.0,0,ESPN,D,NYM,0,03fda22a-f8a3-45e8-80dc-bf8190a22020,2,41922,NE,Citi Field,outdoor,grass,69.0,Cloudy,31.0,49.0,50.0,W,14.0,NaN,0.0,0.0,1.000000,0.000000,6
3,MIN,1,NaN,56da7cb9-5975-46cd-8c63-96fce1245945,0.0,0.0,1,MASN,D,BAL,1,cef2e01f-1349-4e31-bf1e-757ae8cb6bc6,1,45971,NE,Oriole Park at Camden Yards,outdoor,grass,NaN,unknown,NaN,NaN,NaN,unknown,NaN,NaN,0.0,0.0,0.500000,0.500000,7
4,PIT,0,NaN,49f94f4e-c83f-4060-81d5-05ee70d61566,0.0,0.0,2,FS-DET,N,DET,2,f33fc95b-a775-4887-984c-a98494fda8fb,0,41297,SE,Comerica Park,outdoor,grass,89.0,Rain,29.0,58.0,43.0,SSW,16.0,NaN,0.0,0.0,0.000000,1.000000,14
5,BOS,1,NaN,3d6c9984-b764-4d81-bca0-f0fcdcdbd5f1,0.0,0.0,2,SUN,D,TB,2,769c18e7-cb4b-487c-b21a-64de1233d6bd,1,42735,NE,Tropicana Field,indoor,turf,20.0,Cloudy,53.0,55.0,70.0,ESE,9.0,NaN,0.0,0.0,0.333333,0.666667,3
6,SF,1,NaN,cdd52cf2-b10d-456a-9a92-1d74e1f05bac,0.0,0.0,2,ESPN,N,LAD,2,d9936d13-479d-43e0-a845-c74e17046622,1,56000,NE,Dodger Stadium,outdoor,grass,NaN,unknown,NaN,NaN,NaN,unknown,NaN,NaN,0.0,0.0,0.333333,0.666667,9


In [6]:
with open('model_data', 'wb') as fp:
    pickle.dump(data, fp)

In [7]:
data.isnull().sum(axis = 0)

game.away.abbr                           0
game.away.loss                           0
game.away.probable_pitcher.era          85
game.away.probable_pitcher.id            0
game.away.probable_pitcher.loss          0
game.away.probable_pitcher.win           0
game.away.win                            0
game.broadcast.network                   0
game.day_night                           0
game.home.abbr                           0
game.home.loss                           0
game.home.probable_pitcher.id            0
game.home.win                            0
game.venue.capacity                      0
game.venue.field_orientation             0
game.venue.name                          0
game.venue.stadium_type                  0
game.venue.surface                       0
game.weather.forecast.cloud_cover        8
game.weather.forecast.condition          0
game.weather.forecast.dew_point_f        8
game.weather.forecast.humidity           8
game.weather.forecast.temp_f             8
game.weathe